In [1]:
import pinocchio as pin
from pinocchio import casadi as cpin
import casadi
import numpy as np
import example_robot_data as robex
#import matplotlib.pyplot as plt; plt.ion()
from pinocchio.visualize import GepettoVisualizer
from utils.meshcat_viewer_wrapper import  MeshcatVisualizer, colors
import time

In [2]:
# Load the model both in pinocchio and pinocchio casadi
robot = robex.load('talos')
model = robot.model
cmodel = cpin.Model(robot.model)
data = model.createData()
cdata = cmodel.createData()

In [3]:
# Start the Meshcat viewer
viz = MeshcatVisualizer(robot)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [4]:
viz.display(robot.q0)

In [5]:
# reference configuration
q0 = robot.q0
v0 = np.zeros(robot.nv)
a0 = np.zeros(robot.nv)
x0 = np.concatenate([q0, v0])

In [6]:
nq = cmodel.nq
nv = cmodel.nv
nx = nq + nv
ndx = nv *2

cx = casadi.SX.sym('cx', nx, 1)
cdx = casadi.SX.sym('cx', ndx, 1)
ca = casadi.SX.sym('ca', nv, 1)
xref = casadi.SX.sym('cx', nx, 1)
R = casadi.SX.sym('R', 3, 3)
R_ref = casadi.SX.sym('R_ref', 3, 3)

# Get the index of the frames which are going to be used
IDX_BASE = cmodel.getFrameId('torso_2_link')
IDX_LF = cmodel.getFrameId('leg_left_6_link')
IDX_RF = cmodel.getFrameId('leg_right_6_link')
IDX_LG = cmodel.getFrameId('gripper_left_base_link')
IDX_RG = cmodel.getFrameId('gripper_right_base_link')
IDX_LE = cmodel.getFrameId('arm_left_3_joint')
IDX_RE = cmodel.getFrameId('arm_right_3_joint')

# This is needed in order to go from a configuration and the displacementto the final configuration
# Pinocchi.integrate is needed because q and v have different dimensions
integrate = casadi.Function('integrate', [cx, cdx], [ casadi.vertcat(cpin.integrate(cmodel,cx[:nq],cdx[:nv]),
                                                        cx[-nv:]+cdx[-nv:]) ] )

# Casadi function to map states to COM position                                                        
com_position = casadi.Function('com', [cx], [cpin.centerOfMass(cmodel, cdata, cx[: nq])] )

# Forward kinematics and store the data in 'cdata'
# Note that now cdata is filled with symbols, so there is no need to compute the forward kinematics at every variation of q
# Since everything is a symbol, a substituition (which is what casadi functions do) is enough
cpin.forwardKinematics(cmodel, cdata, cx[: nq], cx[nq :], ca)
cpin.updateFramePlacements(cmodel, cdata)

base_rotation = casadi.Function('com', [cx], [cdata.oMf[IDX_BASE].rotation] )

# Casadi functions can't output a SE3 element, so the oMf matrices are split in rotational and translational components

lf_position = casadi.Function('lf_pos', [cx], [cdata.oMf[IDX_LF].translation])
lf_rotation = casadi.Function('lf_rot', [cx], [cdata.oMf[IDX_LF].rotation])
rf_position = casadi.Function('rf_pos', [cx], [cdata.oMf[IDX_RF].translation])
rf_rotation = casadi.Function('rf_rot', [cx], [cdata.oMf[IDX_RF].rotation])

lg_position = casadi.Function('lg_pos', [cx], [cdata.oMf[IDX_LG].translation])
lg_rotation = casadi.Function('lg_rot', [cx], [cdata.oMf[IDX_LG].rotation])
le_rotation = casadi.Function('le_rot', [cx], [cdata.oMf[IDX_LE].rotation])
le_translation = casadi.Function('le_pos', [cx], [cdata.oMf[IDX_LE].translation])

rg_position = casadi.Function('rg_pos', [cx], [cdata.oMf[IDX_RG].translation])
rg_rotation = casadi.Function('rg_rot', [cx], [cdata.oMf[IDX_RG].rotation])
re_rotation = casadi.Function('re_rot', [cx], [cdata.oMf[IDX_RE].rotation])
re_translation = casadi.Function('re_pos', [cx], [cdata.oMf[IDX_RE].translation])

log = casadi.Function('log', [R, R_ref], [cpin.log3(R.T @ R_ref)])


In [7]:
# Defining weights

parallel_cost = 1e3
distance_cost = 1e3
straightness_body_cost = 1e3
distance_btw_hands = 0.3

opti = casadi.Opti()

# Note that here the optimization variables are Dq, not q, and q is obtained by integrating.
# q = q + Dq, where the plus sign is intended as an integrator (because nq is different from nv)
# It is also possible to optimize directly in q, but in that case a constraint must be added in order to have 
# the norm squared of the quaternions = 1
dxs = opti.variable(ndx)
xs = integrate(x0, dxs)

kle = le_rotation(xs)@np.array([1,0,0])
kre = re_rotation(xs)@np.array([1,0,0])


cost = casadi.sumsqr(xs - x0)

# Distance between the hands
cost += distance_cost * casadi.sumsqr(lg_position(xs) - rg_position(xs) - np.array([0, distance_btw_hands, 0]))  

# Cost on parallelism of the two hands
r_ref = pin.utils.rotate('x', 3.14 / 2) # orientation target
cost += parallel_cost * casadi.sumsqr(log(rg_rotation(xs), r_ref))
r_ref = pin.utils.rotate('x', -3.14 / 2) # orientation target
cost += parallel_cost * casadi.sumsqr(log(lg_rotation(xs), r_ref))

# Body in a straight position
cost += straightness_body_cost * casadi.sumsqr(log(base_rotation(xs), base_rotation(x0)))

cost +=  -1000 * casadi.sumsqr(le_translation(xs)[1] - re_translation(xs)[1])

opti.minimize(cost)

# COM
opti.subject_to(opti.bounded(-0.1, com_position(xs)[0], 0.1))
opti.subject_to(opti.bounded(-0.02, com_position(xs)[1], 0.02))
opti.subject_to(opti.bounded(0.7, com_position(xs)[2], 0.9))

# Standing foot
opti.subject_to(rf_position(xs) - rf_position(x0) == 0)
opti.subject_to(rf_rotation(xs) - rf_rotation(x0) == 0)

# Free foot
opti.subject_to(lf_position(xs)[2] >= 0.4)
opti.subject_to(opti.bounded(0.05, lf_position(xs)[0:2], 0.1))

r_ref = pin.utils.rotate('z', 3.14 / 2) @ pin.utils.rotate('y', 3.14 / 2) # orientation target
opti.subject_to(opti.bounded(-0.0, lf_rotation(xs) - r_ref, 0.0))

# Left hand constraint to be at a certain height
opti.subject_to(opti.bounded(1.1, rg_position(xs)[2], 1.2))
opti.subject_to(opti.bounded(-distance_btw_hands/2, rg_position(xs)[1], 0))


In [8]:
def call(xs):
    xs_tmp = integrate(x0, opti.debug.value(dxs)).full()
    viz.display(xs_tmp[: nq])

In [9]:
# Pretty crude, but it shows what's going on
# It becomes very slow, so just for seeing

opti.callback(call)

In [13]:
# Solve the oiptimization problem

opti.solver('ipopt')
opti.set_initial(dxs, np.zeros(ndx))
opti.solve()
print("Final cost is: ", opti.value(cost))

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      186
Number of nonzeros in inequality constraint Jacobian.:      174
Number of nonzeros in Lagrangian Hessian.............:      408

Total number of variables............................:       76
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       21
Total number of inequality constraints...............:        8
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        7
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [11]:
x_sol = integrate(x0, opti.value(dxs)).full()
q_sol = x_sol[: nq]
v_sol = x_sol[nq:]

In [12]:
viz.display(q_sol)